<a href="https://colab.research.google.com/github/AUT-Student/BigData-Project/blob/main/BigData_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><b>In the name of God</b></center>

<b>Course</b>: ‌Big Data
<br>
<b>Description:</b> Final Project
<br>
<b>Developers</b>: 
* Seyyed Hossein Khatami (400131077)
* Alireza Mazochi (400131075)

# Library

In [21]:
from abc import ABC, abstractmethod
from dataclasses import dataclass
import matplotlib.pyplot as plt
from copy import deepcopy
import pandas as pd
import numpy as np
import random
import heapq
import math
import glob
import time

In [43]:
DEBUG = True

# Load Datasets

## CAIDA

In [3]:
!gdown https://snap.stanford.edu/data/as-caida.tar.gz
!mkdir caida
!tar -C /content/caida -xzf /content/as-caida.tar.gz 

Downloading...
From: https://snap.stanford.edu/data/as-caida.tar.gz
To: /content/as-caida.tar.gz
100% 46.4M/46.4M [00:03<00:00, 13.5MB/s]
mkdir: cannot create directory ‘caida’: File exists


## Oregon

In [4]:
!rm oregon -r

In [5]:
!mkdir oregon
%cd oregon

!gdown https://snap.stanford.edu/data/oregon1_010331.txt.gz
!gdown https://snap.stanford.edu/data/oregon1_010407.txt.gz
!gdown https://snap.stanford.edu/data/oregon1_010414.txt.gz
!gdown https://snap.stanford.edu/data/oregon1_010421.txt.gz
!gdown https://snap.stanford.edu/data/oregon1_010428.txt.gz
!gdown https://snap.stanford.edu/data/oregon1_010505.txt.gz
!gdown https://snap.stanford.edu/data/oregon1_010512.txt.gz
!gdown https://snap.stanford.edu/data/oregon1_010519.txt.gz
!gdown https://snap.stanford.edu/data/oregon1_010526.txt.gz


!gunzip oregon1_010331.txt.gz
!gunzip oregon1_010407.txt.gz
!gunzip oregon1_010414.txt.gz
!gunzip oregon1_010421.txt.gz
!gunzip oregon1_010428.txt.gz
!gunzip oregon1_010505.txt.gz
!gunzip oregon1_010512.txt.gz
!gunzip oregon1_010519.txt.gz
!gunzip oregon1_010526.txt.gz

%cd ..

/content/oregon
Downloading...
From: https://snap.stanford.edu/data/oregon1_010331.txt.gz
To: /content/oregon/oregon1_010331.txt.gz
100% 69.1k/69.1k [00:00<00:00, 342kB/s]
Downloading...
From: https://snap.stanford.edu/data/oregon1_010407.txt.gz
To: /content/oregon/oregon1_010407.txt.gz
100% 69.3k/69.3k [00:00<00:00, 337kB/s]
Downloading...
From: https://snap.stanford.edu/data/oregon1_010414.txt.gz
To: /content/oregon/oregon1_010414.txt.gz
100% 69.8k/69.8k [00:00<00:00, 341kB/s]
Downloading...
From: https://snap.stanford.edu/data/oregon1_010421.txt.gz
To: /content/oregon/oregon1_010421.txt.gz
100% 70.7k/70.7k [00:00<00:00, 346kB/s]
Downloading...
From: https://snap.stanford.edu/data/oregon1_010428.txt.gz
To: /content/oregon/oregon1_010428.txt.gz
100% 70.1k/70.1k [00:00<00:00, 348kB/s]
Downloading...
From: https://snap.stanford.edu/data/oregon1_010505.txt.gz
To: /content/oregon/oregon1_010505.txt.gz
100% 70.5k/70.5k [00:00<00:00, 344kB/s]
Downloading...
From: https://snap.stanford.edu/d

# Data Structure Classes

## Edge Class

In [6]:
@dataclass(init=True, eq=True, frozen=True)
class Edge:
  source: int
  destination: int

  def get_reverse(self):
    return Edge(source=self.destination, destination=self.source)

  def get_minmax(self):
    return Edge(source=min(self.source, self.destination),
                destination=max(self.source, self.destination))

## Graph Class

In [7]:
class Graph():
  def __init__(self, directed):
    self.edges = set()
    self.nodes = set()
    self.node_order = list()
    self.node_order_dictionary = dict()

    self.reset_statics()

  def _set_order(self, node):
    if node not in self.node_order_dictionary:
      self.node_order_dictionary[node] = len(self.node_order)
      self.node_order.append(node)

  def get_order(self, node):
    return self.node_order_dictionary[node]

  def add_edge(self, edge:Edge):
    self.nodes.add(edge.source)
    self.nodes.add(edge.destination)
    self.edges.add(edge)

    self._set_order(edge.source)

  def finish(self):
    for node in self.nodes:
      # Set order for remaining nodes
      self._set_order(node)

    self.reset_statics()

  def reset_statics(self):
    self.number_trees = None
    self.node_adjacency_list = None

  def number_nodes(self):
    return len(self.nodes)
  
  def number_edges(self):
    return len(self.edges)

  def get_edges(self):
    return self.edges
  
  def get_nodes(self):
    return self.nodes

  def get_node(self, node_id):
    return self.node_order[node_id]
  
  def get_node_adjacency_list(self, node_id):
    if self.node_adjacency_list is None:
      self.calculate_node_adjacency_list()
    
    return self.node_adjacency_list[self.get_node(node_id)]

  def calculate_node_adjacency_list(self):
    self.node_adjacency_list = dict()

    for node in self.nodes:
      self.node_adjacency_list[node] = set()
    
    for edge in self.edges:
      self.node_adjacency_list[edge.source].add(edge.destination) 
      self.node_adjacency_list[edge.destination].add(edge.source) 
    
  def calculate_number_trees(self):
    if self.number_trees is not None:
      return self.number_trees

    self.number_trees = 0
    for node_x in self.nodes:
      for node_y in self.node_adjacency_list[node_x]:
        for node_z in self.node_adjacency_list[node_x]:

          if node_x < node_y < node_z:
            if node_z in self.node_adjacency_list[node_y]:
              self.number_trees += 1

  def get_number_trees(self):
    if self.number_trees is None:
      self.calculate_number_trees()

    return self.number_trees

  def get_N_edge(self, edge):
    node_x = edge.source
    node_y = edge.destination

    number_trees = 0

    for node_z in self.node_adjacency_list[node_x]:
      if node_z in self.node_adjacency_list[node_y]:
        number_trees += 1
    
    return number_trees

  def get_R_edge(self, edge):
    node_x = edge.source
    node_y = edge.destination

    node_x_order = self.get_order(node_x)
    node_y_order = self.get_order(node_y)

    if node_x_order > node_y_order:
      return 0

    number_trees = 0

    for node_z in self.node_adjacency_list[node_x]:
      if node_z in self.node_adjacency_list[node_y]:
        if node_x_order < self.get_order(node_z) < node_y_order:
          number_trees += 1
          
    return number_trees

## Snapshot

In [8]:
class Snapshot():
  def __init__(self, dataset_path):
    self.dataset_path = dataset_path
    self.graphs = list()

  def load_graphs(self):
    for path in sorted(glob.glob(pathname=self.dataset_path, recursive=True)):
      print(".", end="")
      graph = Graph(directed=False)

      with open(path, "r") as file:
        for i, line in enumerate(file.readlines()):
          if line[0] == "#": continue
          
          source, destination = line[:-1].split("\t")[:2]
          if source == destination: continue
          edge = Edge(source=int(source), destination=int(destination))
          graph.add_edge(edge)
        
      graph.finish()
      graph.calculate_node_adjacency_list()
      self.graphs.append(graph)

  def get_graph(self, i):
    return self.graphs[i]
  
  def number_graphs(self):
    return len(self.graphs)

# Load Dataset

## Oregon

In [9]:
oregon = Snapshot(dataset_path = "/content/oregon/*")

In [10]:
oregon.load_graphs()

.........

In [11]:
oregon.number_graphs()

9

## Caida 2006

In [12]:
caida2006 = Snapshot(dataset_path = "/content/caida/as-caida2006*")

In [13]:
caida2006.load_graphs()

....................................................

In [14]:
caida2006.number_graphs()

52

## Caida 2007

In [15]:
caida2007 = Snapshot(dataset_path = "/content/caida/as-caida2007*")

In [16]:
caida2007.load_graphs()

..............................................

In [17]:
caida2007.number_graphs()

46

# Oracles

## Heavy Edge Oracle

In [18]:
class HeavyEdgeOracle():
  def __init__(self, graph:Graph):
    self.graph = graph
    self.rho = None
    self.T = None
    self.N_edge = None

  def set_rho(self, rho):
    self.rho = rho

  def train(self):
    if DEBUG: print("Cacluate total trees...")
    self.graph.calculate_number_trees()
    self.T = self.graph.get_number_trees()
    self._calculate_N_edge()
    if DEBUG: print("Training oracle completed!")

  def _calculate_N_edge(self):
    N_edge_complete = list()

    if DEBUG: print("Counting trees per edges (N_e)...")
    edges = self.graph.get_edges()
    for i, edge in enumerate(edges):
      if DEBUG and i % (len(edges)//10)==0:
        print(f"{round(i*100/len(edges), 0)}%", end=" ")
      if edge.source < edge.destination:
        number_trees = self.graph.get_N_edge(edge)
        
        N_edge_complete.append({"edge": edge, "number_trees": number_trees})

    if DEBUG: print("\nSorting and selecting top N_e")
    N_edge_top = sorted(N_edge_complete, key=lambda x:-x["number_trees"])[:len(N_edge_complete)//10]

    self.N_edge = dict()
    for item in N_edge_top:
      self.N_edge[item["edge"]] = item["number_trees"]

  def is_heavy(self, edge:Edge):
    edge = edge.get_minmax()
    if edge in self.N_edge:
      return self.N_edge[edge] >= self.T/self.rho
    else:
      return False

## Value Oracle

In [19]:
class ValueOracle():
  def __init__(self, graph: Graph):
    self.graph = graph
    
    self.alpha = 1
    self.beta = 10
    self.K = 1
    self.T = None
    self.N_edge = None


  def train(self):
    if DEBUG: print("Cacluate total trees...")
    self.graph.calculate_number_trees()
    self.T = self.graph.get_number_trees()
    self._calculate_N_edge()
    if DEBUG: print("Training oracle completed!")

  def _calculate_N_edge(self):
    N_edge_complete = list()

    if DEBUG: print("Counting trees per edges (N_e)...")
    edges = self.graph.get_edges()
    for i, edge in enumerate(edges):
      if DEBUG and i % (len(edges)//10)==0:
        print(f"{round(i*100/len(edges), 0)}%", end=" ")
      if edge.source < edge.destination:
        number_trees = self.graph.get_N_edge(edge)
        
        N_edge_complete.append({"edge": edge, "number_trees": number_trees})

    if DEBUG: print("\nSorting and selecting top N_e")
    N_edge_top = sorted(N_edge_complete, key=lambda x:-x["number_trees"])[:len(N_edge_complete)//10]

    self.N_edge = dict()
    for item in N_edge_top:
      self.N_edge[item["edge"]] = item["number_trees"]

  def estimate_N(self, edge:Edge):
    return self.N_edge.get(edge.get_minmax(), 0) 

In [20]:
# class ValueOracle():
#   def __init__(self, graph: Graph):
#     self.graph = graph
    
#     self.alpha = 1
#     self.beta = 10
#     self.K = 1
#     self.T = None
  
#   def train(self):
#     if DEBUG: print("Cacluate total trees...")
#     self.graph.calculate_number_trees()
#     self.T = self.graph.get_number_trees()
#     self._calculate_N_edge()
#     if DEBUG: print("Training oracle completed!")

#   def _calculate_N_edge(self):
#     R_edge_complete = list()

#     if DEBUG: print("Counting Ordered trees per edges (R_e)...")
#     edges = self.graph.get_edges()
#     for i, edge in enumerate(edges):
#       if DEBUG and i % (len(edges)//10)==0:
#         print(f"{round(i*100/len(edges), 0)}%", end=" ")
#       if self.graph.get_order(edge.source) < self.graph.get_order(edge.destination):
#         number_trees = self.graph.get_R_edge(edge)
        
#         if number_trees > 0:
#           R_edge_complete.append({"edge": edge, "number_trees": number_trees})

#     if DEBUG: print("\nSorting R_e")
#     R_edge_complete = sorted(R_edge_complete, key=lambda x:-x["number_trees"])

#     self.R_edge = dict()
#     for item in R_edge_complete:
#       self.R_edge[item["edge"]] = item["number_trees"]

#   def estimate_R(self, edge:Edge):
#     return self.R_edge.get(edge, 0)

# Estimators

In [59]:
class Estimator(ABC):
  @abstractmethod
  def estimate_number_trees(self):
    pass

  @abstractmethod
  def number_stored_edges(self):
    pass

## Algorithm 1

In [23]:
class EstimatorAlgorithm1(Estimator):
  def __init__(self, graph:Graph, oracle:HeavyEdgeOracle, epsilon,
               alpha, beta, gamma):
    self.graph = graph 
    self.oracle = oracle

    self.epsilon = epsilon
    self.alpha = alpha
    self.gamma = gamma
    self.beta = beta

    self.A_l, self.A_m, self.A_h = 0, 0, 0
    self.S_l, self.S_m, self.S_aux = set(), set(), set()

    self.T = self.oracle.T

    self.C = dict()
    self.B = dict()

    self.n = graph.number_nodes()
    self.m = graph.number_edges()

    self.set_probabilities()

  def set_probabilities(self):
    if self.T >= ((self.m/self.epsilon) ** 0.5):
      self.rho = (self.m * self.T) ** (1/3)      
      
      self.p1 = self.alpha * (self.epsilon ** -2) / self.rho
      self.p2 = min(self.beta*(self.epsilon ** -2) * self.rho / self.T , 1)
      self.p3 = self.gamma * (self.epsilon ** (-2)) * math.log(self.n) / self.rho
    
    else:
      self.rho = (self.m ** 0.5) / self.epsilon
      self.p1 = 0
      self.p2 = 1
      self.p3 = self.gamma * (self.epsilon ** (-2)) / self.rho
    
    if DEBUG:
      print(f"T = {self.T}, rho = {self.rho}, T/rho = {self.T//self.rho}")
      print(f"p1 = {self.p1}, p2 = {self.p2}, p3 = {self.p3}")
    self.oracle.set_rho(self.rho)

  def estimate_number_trees(self):
    for node_id in range(self.graph.number_nodes()):
      if DEBUG and node_id%(self.graph.number_nodes()//10)==0:
         print(f"{round(node_id*100/self.graph.number_nodes())}%", end=" ")
      v = self.graph.get_node(node_id)
      N_v = self.graph.get_node_adjacency_list(node_id)

      for ab in self.S_l.union(self.S_m):
        if ab.source in N_v and ab.destination in N_v:
          self.C[ab] += 1
      
      for a in N_v:
        av = Edge(a, v)

        if av in self.S_aux:
          self.B[av] = 1

      for u in N_v:
        vu = Edge(v, u)
        uv = Edge(u, v)
        
        if random.random() < self.p3:
          self.S_aux.add(vu)
          self.B[vu] = 0

        if self.oracle.is_heavy(uv) is False:
          if uv in self.S_l:
            self.A_l += self.C[uv]
          elif random.random() < self.p1:
            self.S_l.add(vu)
            self.C[vu] = 0
        else:
          sharp = 0
          for edge in self.S_aux:
            if edge.source == u and self.B[edge]==1 and edge.destination in N_v:
              sharp += 1
          
          if sharp >= self.p3 * self.rho:
            self.A_h += sharp
          elif uv in self.S_m:
            self.A_m += self.C[uv]
          elif random.random() < self.p2:
            self.S_m.add(vu)
            self.C[vu] = 0
    
    if DEBUG:
      print(f"Len(S_l) = {len(self.S_l)}, Len(S_m) = {len(self.S_m)}, Len(S_aux)={len(self.S_aux)}")
      print(f"A_l {self.A_l}, A_m = {self.A_m}, A_h = {self.A_h}")
    return self.A_l / self.p1 + self.A_m / self.p2 + self.A_h / self.p3

  def number_stored_edges(self):
    return len(self.S_l) + len(self.S_m) + len(self.S_aux)

## Algorithm 2

In [24]:
class EstimatorAlgorithm2(Estimator):
  def __init__(self, oracle: ValueOracle, graph:Graph
               , epsilon, c, H_constant=1):
    
    self.oracle = oracle
    self.graph = graph
    self.epsilon = epsilon
    self.c = c

    self.K = self.oracle.K
    self.T = self.oracle.T
    self.m = self.graph.number_edges()
    self.alpha = self.oracle.alpha
    self.beta = self.oracle.beta

    self.H = int( 1/(self.epsilon**2)\
                  * (math.log(self.K/self.epsilon ,10) ** 2)\
                  * (self.alpha + self.m*self.beta/self.T)\
                  * H_constant)
    
    self.estimation_numbers = int(self.c * (self.epsilon ** -2))

    if DEBUG: print(f"H = {self.H}, estimation_numbers = {self.estimation_numbers}")

    self.X = list()
    self.S = list()
    self.Q = list()
    self.A = list()
    self.C = list()
    self.R_div_u = list()

  def estimate_number_trees(self):
    M = 0
    c_plus_counter = 0
    for _ in range(self.estimation_numbers):
      self.S.append(set())
      self.A.append(0)
      self.C.append(dict())
      self.Q.append(dict())

    for node_id in range(self.graph.number_nodes()):
      if DEBUG and node_id%(self.graph.number_nodes()//10)==0 :
         print(f"{round(node_id*100/self.graph.number_nodes())}%")
         print(f"M = {M}")
         print(f"""Top A = {sorted([int(Ai) for Ai in self.A], reverse=True)[:10]},
          Median A = {sorted([int(Ai) for Ai in self.A], reverse=True)[self.estimation_numbers//2]}""")
         print(f"Top len(S) = {sorted([len(Si) for Si in self.S], reverse=True)[:10]}")
         print(f"C Plus Counter = {c_plus_counter}")
         c_plus_counter = 0

      y = self.graph.get_node(node_id)
      N_y = self.graph.get_node_adjacency_list(node_id)

      for i in range(self.estimation_numbers):
        for edge in self.S[i]:
          if edge.source in N_y and edge.destination in N_y:
            self.C[i][edge] += 1
            c_plus_counter += 1

      for x in N_y:
        yx = Edge(y, x)
        for i in range(self.estimation_numbers):
          xy = Edge(x, y)
          if xy in self.S[i]:
            self.A[i] = max(self.A[i], self.C[i][xy]/self.Q[i][xy])
          else:
            R_yx_hat = self.oracle.estimate_N(yx)
            R_yx_hat += self.beta

            self.Q[i][yx] = np.random.exponential(1)
            self.C[i][yx] = 0
            self.S[i].add(yx)

            heapq.heappush(self.R_div_u, (-R_yx_hat/self.Q[i][yx], {"edge": yx, "i": i}))
      
      sum_s = 0
      M = np.inf
      for (s, _) in self.R_div_u:
        sum_s += 1

        if sum_s > self.H:
          break
        else:
          M = -s
      
      reduced_R_div_u = list()

      R_div_u_length = len(self.R_div_u) 
      for _ in range(R_div_u_length):
        (s, edge_i) = heapq.heappop(self.R_div_u)
        edge, i = edge_i["edge"], edge_i["i"]
        
        if -s >= M:
          heapq.heappush(reduced_R_div_u, (s, edge_i))
        else:
          self.Q[i].pop(edge)
          self.C[i].pop(edge)
          self.S[i].remove(edge)
      
      self.R_div_u = reduced_R_div_u

    for i in range(self.estimation_numbers):
      self.X.append(self.A[i])

    self.X = sorted(self.X)
    if DEBUG: print(f"Top X = {self.X[-10:]}")
    X_median = self.X[len(self.X)//2]

    return int(math.log(2) * X_median)

  def number_stored_edges(self):
    return  sum([len(Si) for Si in self.S])

## Algorithm 3

In [25]:
class EstimatorAlgorithm3(Estimator):
  def __init__(self, oracle:ValueOracle, graph:Graph, epsilon,
               i_constant, j_constant, c):
    
    self.oracle = oracle
    self.graph = graph
    self.epsilon = epsilon
    self.c = c
    self.n = self.graph.number_nodes()
    self.beta = self.oracle.beta
    self.T = self.oracle.T

    self.i = math.ceil(math.log(self.n, 10)) * i_constant
    self.j = math.ceil(math.log(math.log(self.n, 10), 10)) * j_constant
    
    self.p = [None] * self.i
    self.I = [(None, None)] * self.i

    self.p[0] = self.c * (self.epsilon ** -2) * 1 * self.beta / self.T
    self.I[0] = (0, 2*self.beta)

    for ii in range(1, self.i):
      self.p[ii] = self.c * (self.epsilon ** -2) * (2 ** ii) \
                          * self.beta * (math.log(self.n, 10) ** 2) / self.T
      self.I[ii] = ((2**ii)*self.beta, (2**(ii+1))*self.beta)

    self.H = 0
    self.S = list()
    self.A = list()

    for ii in range(self.i):
      S_list_j = []
      A_list_j = []
      for jj in range(self.j):
        S_list_j.append(set())
        A_list_j.append(0)

      self.S.append(S_list_j)
      self.A.append(A_list_j)

    self.C = dict()

    if DEBUG: 
      print(f"i = {self.i}, j = {self.j}")
      for ii in range(self.i):
        print(f"p({ii}) = {self.p[ii]}")
      
      for ii in range(self.i):
        print(f"I({ii}) = {self.I[ii]}")

  def estimate_number_trees(self):
    for node_id in range(self.graph.number_nodes()):
      if DEBUG and node_id%(self.graph.number_nodes()//10)==0:
         print(f"{round(node_id*100/self.graph.number_nodes())}%", end=" ")

      y = self.graph.get_node(node_id)
      N_y = self.graph.get_node_adjacency_list(node_id)

      for ii in range(self.i):
        for jj in range(self.j):
          for edge in self.S[ii][jj]:
            if edge.source in N_y and edge.destination in N_y:
              self.C[edge] += 1

      for x in N_y:
        xy = Edge(x, y)
        yx = Edge(y, x)

        flag = False
        for ii in range(self.i):
          for jj in range(self.j):
            if xy in self.S[ii][jj]:
              self.A[ii][jj] += self.C[xy] # Changing original sudocode
              flag = True

        if flag is False:
          R_yx_hat = self.oracle.estimate_N(yx)

          selected_i = None
          for ii in range(self.i):
            if self.I[ii][0] <= R_yx_hat + self.beta < self.I[ii][1]:
              selected_i = ii
          
          for jj in range(self.j):
            if random.random() < self.p[selected_i]:
              self.S[selected_i][jj].add(yx)
              self.C[yx] = 0

    for ii in range(self.i):
      A_ij_median = sorted(self.A[ii])[self.j//2]
      self.H += A_ij_median/self.p[ii]
    
    if DEBUG: 
      print(f"\nS = {[[len(Sij) for Sij in Si] for Si in self.S]}")
      print(f"A = {self.A}")
    
    return int(self.H)

  def number_stored_edges(self):
    return sum([sum([len(Sij) for Sij in Si]) for Si in self.S])

## Algorithm 4

In [73]:
class EstimatorAlgorithm4(Estimator):
  def __init__(self, graph:Graph, oracle:HeavyEdgeOracle, epsilon, C):
    self.graph = graph
    self.oracle = oracle
    self.C = C
    self.epsilon = epsilon
    self.T = self.oracle.T
    self.m = self.graph.number_edges()

    self.rho = max(self.epsilon*self.T/math.sqrt(self.m), 1)
    self.p = C * max(1 / (self.epsilon * math.sqrt(self.T)),
                     self.rho/ (self.epsilon**2 * self.T))

    self.oracle.set_rho(self.rho)

    self.l1, self.l2, self.l3 = 0, 0, 0
    self.S_L = set()
    self.H = set()

    self.stored_nodes = set()

    if DEBUG: print(f"rho = {self.rho}, p = {self.p}")

  def estimate_number_trees(self):
    shuffled_edges = deepcopy(list(self.graph.get_edges()))
    random.shuffle(shuffled_edges)

    for i, edge in enumerate(shuffled_edges):
      if DEBUG and i%(len(shuffled_edges)//10)==0:
        print(f"{round(i*100/len(shuffled_edges))}%", end=" ")

      edge = edge.get_minmax()
      a = edge.source
      b = edge.destination

      if self.oracle.is_heavy(edge):
        self.H.add(edge)
        self.stored_nodes.add(a)
        self.stored_nodes.add(b)
      else:
        if random.random() < self.p:
          self.S_L.add(edge)
          self.stored_nodes.add(a)
          self.stored_nodes.add(b)

      for v in self.stored_nodes:
        if v != a and v != b:
          av = Edge(a, v).get_minmax()
          bv = Edge(b, v).get_minmax()

          if av in self.S_L and bv in self.S_L:
            self.l1 += 1
          elif av in self.S_L and bv in self.H:
            self.l2 += 1
          elif av in self.H and bv in self.S_L:
            self.l2 += 1
          elif av in self.H and bv in self.H:
            self.l3 += 1

    return int(self.l1/(self.p**2) + self.l2/self.p + self.l3)

  def number_stored_edges(self):
    return len(self.S_L) + len(self.H)

# Results

## Test Mesure

In [32]:
def relative_error(real_number_trees, predicted_number_trees):
  return abs(1 - predicted_number_trees/real_number_trees)

## Parameters

### Dataset Parameters

In [49]:
TRAIN_GRAPH_ID = {"Oregon": 0, "CAIDA2006": 0, "CAIDA2007":0}
TEST_GRAPH_ID = {"Oregon":4-1, "CAIDA2006":30-1, "CAIDA2007":25-1}

In [31]:
OREGON_TRAIN_GRAPH = 0
CAIDA2006_TRAIN_GRAPH = 0
CAIDA2007_TRAIN_GRAPH = 0

OREGON_TEST_GRAPH = 4 - 1
CAIDA2006_TEST_GRAPH = 30 - 1
CAIDA2007_TEST_GRAPH = 25 - 1

In [56]:
datasets = {
    "Oregon": oregon,
    "CAIDA2006": caida2006,
    "CAIDA2007": caida2007
}

### Global Parameters

In [69]:
EPSILON = 0.05
NUMBER_RUNS = 1
DEBUG = False

### Algorithms Parameter

In [71]:
ALGORITHM_PARAMETERS = {
    "Algorithm1": {"alpha": 0.1, "beta": 0.5, "gamma": 0.01},
    "Algorithm2": {"H_constant": 0.5, "c":0.2},
    "Algorithm3": {"i_constant": 2, "j_constant": 1, "c": 1},
    "Algorithm4": {"C":1}
}

## Oracle Training

In [63]:
oracles = {}
for dataset_name in ["Oregon", "CAIDA2006", "CAIDA2007"]:
  heavy_edge_oracle = HeavyEdgeOracle(datasets[dataset_name].get_graph(TRAIN_GRAPH_ID[dataset_name]))
  heavy_edge_oracle.train()

  value_oracle = ValueOracle(datasets[dataset_name].get_graph(TRAIN_GRAPH_ID[dataset_name]))
  value_oracle.train()

  oracles[dataset_name] = {"HeavyEdgeOracle": heavy_edge_oracle,
                           "ValueOracle":value_oracle}

In [64]:
def create_estimator(algorithm_name, dataset_name) -> Estimator:
  if algorithm_name=="Algorithm1":
    estimator = EstimatorAlgorithm1(graph=datasets[dataset_name].get_graph(TEST_GRAPH_ID[dataset_name]),
                                    oracle=oracles[dataset_name]["HeavyEdgeOracle"],
                                    epsilon=EPSILON,
                                    alpha=ALGORITHM_PARAMETERS["Algorithm1"]["alpha"],
                                    beta=ALGORITHM_PARAMETERS["Algorithm1"]["beta"],
                                    gamma=ALGORITHM_PARAMETERS["Algorithm1"]["gamma"])
  
  elif algorithm_name=="Algorithm2":
    estimator = EstimatorAlgorithm2(graph=datasets[dataset_name].get_graph(TEST_GRAPH_ID[dataset_name]),
                                    oracle=oracles[dataset_name]["ValueOracle"],
                                    H_constant=ALGORITHM_PARAMETERS["Algorithm2"]["H_constant"],
                                    c=ALGORITHM_PARAMETERS["Algorithm2"]["c"],
                                    epsilon=EPSILON)
  
  elif algorithm_name=="Algorithm3":
    estimator = EstimatorAlgorithm3(graph=datasets[dataset_name].get_graph(TEST_GRAPH_ID[dataset_name]),
                                    oracle=oracles[dataset_name]["ValueOracle"],
                                    i_constant=ALGORITHM_PARAMETERS["Algorithm3"]["i_constant"],
                                    j_constant=ALGORITHM_PARAMETERS["Algorithm3"]["j_constant"],
                                    c=ALGORITHM_PARAMETERS["Algorithm3"]["c"],
                                    epsilon=EPSILON)
  
  elif algorithm_name=="Algorithm4":
    estimator = EstimatorAlgorithm4(graph=datasets[dataset_name].get_graph(TEST_GRAPH_ID[dataset_name]),
                                    oracle=oracles[dataset_name]["HeavyEdgeOracle"],
                                    C=ALGORITHM_PARAMETERS["Algorithm4"]["C"],
                                    epsilon=EPSILON)
  return estimator

## Evaluate

In [ ]:
def evaluate(algorithm_name, dataset_name):
  print(f"Resutls {algorithm_name} on dataset {dataset_name}:")
  storage_list = []
  error_list = []

  start_time = time.time()
  for run in range(NUMBER_RUNS):
    print(f"Run {run+1}")

    estimator = create_estimator(algorithm_name, dataset_name)

    estimated_number_trees = estimator.estimate_number_trees() 
    real_number_trees = datasets[dataset_name].get_graph(TEST_GRAPH_ID[dataset_name]).get_number_trees()
    
    storage = estimator.number_stored_edges()
    error = relative_error(real_number_trees, estimated_number_trees) * 100

    storage_list.append(storage)
    error_list.append(error)

  total_time = time.time() - start_time

  storage_mean = np.average(storage_list)
  error_mean = np.average(error_list)
  error_std = np.std(error_list)
  time_mean = total_time / NUMBER_RUNS

  print(f"Space = {round(storage_mean/1000, 2)} * 10^3")
  print(f"Error = {round(error_mean,2)}% ± {round(error_std,2)}%")
  print(f"Time = {round(time_mean)} s")
  print("==========================================\n")

## Run

In [ ]:
for dataset_name in ["Oregon", "CAIDA2006", "CAIDA2007"]:
  for algorithm_name in ["Algorithm4"]:
    evaluate(algorithm_name, dataset_name)

Resutls Algorithm4 on dataset Oregon:
Run 1
Space = 3.47 * 10^3
Error = 10.28% ± 0.0%
Time = 249 s
Resutls Algorithm4 on dataset CAIDA2006:
Run 1
